# From Detection to Credibility: A Machine Learning Framework for Assessing News Source Reliability



In [1]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm

# Plotting and Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Language Detection packages
# `langdetect` for detecting language
from langdetect import detect as langdetect_detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
# `langid` for an alternative language detection method
from langid import classify as langid_classify

# Text Preprocessing and NLP
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords
# Tokenizing sentences/words
from nltk.tokenize import word_tokenize
# Part-of-speech tagging
from nltk import pos_tag
# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer
import nltk
# Regular expressions for text pattern matching
import re

# Word Cloud generation
from wordcloud import WordCloud

In [2]:
data = pd.read_csv('final_df.csv')

In [3]:
data

,year,month,sentiment,processed_full_review
0,2024,3,Neutral,ok use airlin go singapor london heathrow issu...
1,2024,3,Negative,don give money book paid receiv email confirm ...
2,2024,3,Positive,best airlin world best airlin world seat food ...
3,2024,3,Negative,premium economi seat singapor airlin not worth...
4,2024,3,Negative,imposs get promis refund book flight full mont...
...,...,...,...,...
11513,2021,11,Negative,websit buggi paid first busi class ticket webs...
11514,2021,10,Negative,reduc level qualiti servic fear futur airlin t...
11515,2021,10,Negative,chang would cost usd book ticket singapor airl...
11516,2021,8,Negative,disappoint flight check secur check frankfurt ...


# Basic BERT

BERT is a transformer model that provides powerful pre-trained embeddings for downstream tasks such as fake news classification using text.

### Transformer Architecture
To use BERT in Tensorflow, we utilise the `transformers` library by HuggingFace, which simplifies the process of loading pre-trained BERT models and tokenizers. The transformer uses a mechanism called 'self-attention' to weigh the importance of each word in a sentence relative to others, allowing it to process entire sentences at once instead of word-by-word.

### Bidirectional Context Understanding
BERT reads text in both directions at once which helps BERT to understand the full context of each word in relation to its surrounding words, making it excellent at capturing meaning, nuances, and relationships in language.

### Pretraining Tasks
- **Masked Language Modelling (MLM):** Randomly masks some words in sentences and trains the model to predict them. This task encourages BERT to learn contextual relationships and gain a better understanding of language.

- **Next Sentence Prediction (NSP):** Trains BERT to understand relationships between sentences by predicting whether one sentence naturally follows another. This helps BERT with tasks that require understanding sentence-pairs, like question-answering.

BERT is limited to a maximum input length of 512 tokens.

Fine-tuning BERT usually requires fewer epochs (2-4) and smaller batch sizes (16 or 32) due to memory constraints and pre-trained knowledge.



In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import tensorflow as tf
import numpy as np
import random

# Set random seed for reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Labels mapping
sentiment_dict = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y = data['sentiment'].map(sentiment_dict).values  # Convert sentiments to numeric labels

train_d, val_d, train_labels, val_labels = train_test_split(data['processed_full_review'], y, test_size=0.2, random_state=42)

In [6]:
texts_train = list(train_d)
texts_val = list(val_d)

max_length = 64

In [7]:
tokenized_texts_train = tokenizer(texts_train, padding=True, truncation=True, return_tensors="pt", max_length=max_length)
tokenized_texts_val = tokenizer(texts_val, padding=True, truncation=True, return_tensors="pt", max_length=max_length)

In [8]:
tokenized_texts_train['input_ids'][0]

tensor([  101,  3532, 23488, 24209, 11475,  3775,  2833,  3532, 18151,  2278,
         3532,  2835,  2067,  2234,  2835,  2053, 18151,  2278,  4521,  2435,
         2053, 18064,  3462,  5463,  2729,  3482,  3256,  6949,  2452,  3198,
         2296,  2239,  2151,  2239,  2215,  2028,  2025,  2053,  2300, 14262,
         2615,  2053,  2522, 16020,  5572,  2445,  4983,  3198,  2028,  3730,
         4392, 14262,  2615,  2048,  2093,  2711,  4511,  2025,  2130,  5254,
          102,     0,     0,     0])

In [9]:
tokenized_texts_train['attention_mask'][0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0])

In [10]:
import torch

In [11]:
train_labels = torch.tensor(list(train_labels))
val_labels = torch.tensor(list(val_labels)) 

In [12]:
train_dataset = TensorDataset(tokenized_texts_train['input_ids'], tokenized_texts_train['attention_mask'], train_labels)
val_dataset = TensorDataset(tokenized_texts_val['input_ids'], tokenized_texts_val['attention_mask'], val_labels)

In [13]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3, num_hidden_layers=12, hidden_size=768, output_attentions=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import StepLR

optimizer = AdamW([
    {'params': model.bert.parameters(), 'lr': 5e-6},
    {'params': model.classifier.parameters(), 'lr': 5e-6}
], lr=5e-6)
criterion = torch.nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, prefetch_factor=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, prefetch_factor=2)

In [17]:
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [18]:
train_losses = []
val_losses = []
val_accuracies = []
train_accuracies = []

In [19]:
from tqdm import tqdm

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    tr_correct_preds = 0
    all_tr_labels = []
    all_tr_preds = []

    # Use tqdm to create a progress bar for the training loop
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Training"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        tr_loss = outputs.loss
        train_loss += tr_loss.item()
        tr_loss.backward()

        tr_logits = outputs.logits
        tr_preds = torch.argmax(tr_logits, dim=1)
        tr_correct_preds += torch.sum(tr_preds == labels).item()

        # Collect predictions and true labels
        all_tr_labels.extend(labels.cpu().numpy())
        all_tr_preds.extend(tr_preds.cpu().numpy())

        clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

    scheduler.step()

    # Calculate average training loss and accuracy
    avg_train_loss = train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    train_accuracy = tr_correct_preds / len(train_d)
    train_accuracies.append(train_accuracy)

    # Calculate Precision, Recall, F1 for training
    train_precision = precision_score(all_tr_labels, all_tr_preds, average='weighted')
    train_recall = recall_score(all_tr_labels, all_tr_preds, average='weighted')
    train_f1 = f1_score(all_tr_labels, all_tr_preds, average='weighted')

    # Validation phase with tqdm progress bar
    model.eval()
    val_loss = 0.0
    correct_preds = 0
    all_val_labels = []
    all_val_preds = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation"):
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct_preds += torch.sum(preds == labels).item()

            # Collect predictions and true labels
            all_val_labels.extend(labels.cpu().numpy())
            all_val_preds.extend(preds.cpu().numpy())

    # Calculate average validation loss and accuracy
    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    val_accuracy = correct_preds / len(val_d)
    val_accuracies.append(val_accuracy)

    # Calculate Precision, Recall, F1 for validation
    val_precision = precision_score(all_val_labels, all_val_preds, average='weighted')
    val_recall = recall_score(all_val_labels, all_val_preds, average='weighted')
    val_f1 = f1_score(all_val_labels, all_val_preds, average='weighted')

    # Print metrics
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    print(f"Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
    print(f"Training Precision: {train_precision:.4f}, Training Recall: {train_recall:.4f}, Training F1: {train_f1:.4f}")
    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation Precision: {val_precision:.4f}, Validation Recall: {val_recall:.4f}, Validation F1: {val_f1:.4f}")


Epoch 1/5 - Validation: 100%|██████████| 36/36 [00:12<00:00,  2.83it/s]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Epoch 1/5
Training Loss: 0.7086, Training Accuracy: 0.7149
Training Precision: 0.6519, Training Recall: 0.7149, Training F1: 0.6722
Validation Loss: 0.4681, Validation Accuracy: 0.8329
Validation Precision: 0.7616, Validation Recall: 0.8329, Validation F1: 0.7937


Epoch 2/5 - Validation: 100%|██████████| 36/36 [00:12<00:00,  2.85it/s]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Epoch 2/5
Training Loss: 0.4739, Training Accuracy: 0.8312
Training Precision: 0.7565, Training Recall: 0.8312, Training F1: 0.7905
Validation Loss: 0.4526, Validation Accuracy: 0.8385
Validation Precision: 0.7648, Validation Recall: 0.8385, Validation F1: 0.7985


Epoch 3/5 - Validation: 100%|██████████| 36/36 [00:12<00:00,  2.80it/s]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Epoch 4/5 - Validation: 100%|██████████| 36/36 [00:12<00:00,  2.80it/s]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Epoch 4/5
Training Loss: 0.4622, Training Accuracy: 0.8337
Training Precision: 0.7925, Training Recall: 0.8337, Training F1: 0.7929
Validation Loss: 0.4521, Validation Accuracy: 0.8394
Validation Precision: 0.7644, Validation Recall: 0.8394, Validation F1: 0.7989


Epoch 5/5 - Validation: 100%|██████████| 36/36 [00:13<00:00,  2.71it/s]


Epoch 5/5
Training Loss: 0.4603, Training Accuracy: 0.8373
Training Precision: 0.8299, Training Recall: 0.8373, Training F1: 0.7966
Validation Loss: 0.4521, Validation Accuracy: 0.8394
Validation Precision: 0.7644, Validation Recall: 0.8394, Validation F1: 0.7989



c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
